In [1]:
import numpy as np
import keras
import os
import matplotlib.image as mpimg


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam

from sklearn import preprocessing

os.environ["CUDA_VISIBLE_DEVICES"]="0"

TRAIN_PATH = "/datasets/tmp/aayang/tiny-imagenet-200/train/"
VAL_IMAGES_PATH = "/datasets/tmp/aayang/tiny-imagenet-200/val/images/"
VAL_ANNO_FILE = "/datasets/tmp/aayang/tiny-imagenet-200/val/val_annotations.txt"
TRAINING_IMAGE_SIZE = 64

LOAD_BATCH_SIZE = 200

Using TensorFlow backend.


In [2]:
def load_training_images(start=0, batch_size=200):
    images = [] #np.ndarray(shape=(batch_size*200, 64,64,3))
    labels = []
    file_names = []
    class_num = 0
    
    for class_folder in os.listdir(TRAIN_PATH):
        images_folder = TRAIN_PATH + class_folder + "/images/"
        x = 0
        
        for image_name in os.listdir(images_folder):
            if x < start:
                x += 1
                continue
            
            image_path = os.path.join(images_folder, image_name)
            image = mpimg.imread(image_path)
            
            if image.shape == (TRAINING_IMAGE_SIZE, TRAINING_IMAGE_SIZE, 3):
                images.append(image/256.0)
                labels.append(class_folder)
                file_names.append(image_name)
    
            x += 1      
            #if x%100 == 0: 
            #    print "class", class_num, "-", x
                
            if x > start+batch_size:
                break
        class_num += 1
                        
    return (np.array(images), np.array(labels), np.array(file_names))

def get_one_hot(le, labels):
    labels_encoded = le.transform(labels)
    y = keras.utils.to_categorical(labels_encoded)
    return y
    
def randomize(images, labels, files):
    shuffle_index = np.random.permutation(len(images))
    
    shuffled_images = images[shuffle_index]
    shuffled_labels = labels[shuffle_index]
    shuffled_files  = files[shuffle_index]
    
    return (shuffled_images, shuffled_labels, shuffled_files)

In [3]:
def load_validation_images():
    with open(VAL_ANNO_FILE) as f:
        lines = f.read().splitlines()
        
    labels = []
    images = []
    idx = 0
    
    for line in lines:
        image_att = line.split("\t")
        image_name = image_att[0]
        image_label = image_att[1]
        
        image_path = os.path.join(VAL_IMAGES_PATH, image_name)
        image = mpimg.imread(image_path)
        
        if image.shape == (TRAINING_IMAGE_SIZE, TRAINING_IMAGE_SIZE, 3):
            images.append(image/256.0)
            labels.append(image_label)
            
    return (np.array(images), np.array(labels))

## 3 Layer CNN

In [4]:
shallow = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
shallow.add(Conv2D(64, (3, 3), activation='relu', input_shape=(64,64,3)))
shallow.add(MaxPooling2D(pool_size=(2, 2)))

#shallow.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
#shallow.add(MaxPooling2D(pool_size=(2, 2)))

#shallow.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
#shallow.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
#shallow.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
#shallow.add(MaxPooling2D(pool_size=(2, 2)))

shallow.add(Flatten())
shallow.add(Dense(1024, activation='relu'))
#shallow.add(Dense(1024, activation='relu'))
#shallow.add(Dropout(0.5))
#shallow.add(Dense(256, activation='relu'))
shallow.add(Dense(200, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
shallow.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])

## 6 Layer CNN

In [5]:
six_layer = Sequential()

six_layer.add(Conv2D(64, (3, 3), activation='relu', input_shape=(64,64,3)))
six_layer.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
six_layer.add(MaxPooling2D(pool_size=(2, 2)))

six_layer.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
six_layer.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
six_layer.add(MaxPooling2D(pool_size=(2, 2)))
six_layer.add(Dropout(0.25))

#six_layer.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
#six_layer.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
#six_layer.add(Conv2D(256, (5, 5), activation='relu', padding="same"))
#six_layer.add(MaxPooling2D(pool_size=(2, 2)))

six_layer.add(Flatten())
six_layer.add(Dense(1024, activation='relu'))
#six_layer.add(Dense(1024, activation='relu'))
#six_layer.add(Dropout(0.5))
#six_layer.add(Dense(256, activation='relu'))
six_layer.add(Dense(200, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
six_layer.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])

## 9 Layer CNN

In [6]:
nine_layer = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
nine_layer.add(Conv2D(64, (3, 3), activation='relu', input_shape=(64,64,3)))
nine_layer.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
nine_layer.add(MaxPooling2D(pool_size=(2, 2)))

nine_layer.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
nine_layer.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
nine_layer.add(MaxPooling2D(pool_size=(2, 2)))
nine_layer.add(Dropout(0.25))

nine_layer.add(Conv2D(256, (3, 3), activation='relu', padding="same"))
nine_layer.add(Conv2D(256, (3, 3), activation='relu', padding="same"))
nine_layer.add(MaxPooling2D(pool_size=(2, 2)))

nine_layer.add(Flatten())
nine_layer.add(Dense(1024, activation='relu'))
nine_layer.add(Dense(1024, activation='relu'))
nine_layer.add(Dropout(0.5))
nine_layer.add(Dense(200, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
nine_layer.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])

## Training

In [7]:
train_le = None
NUM_EPOCHS = 10

for start_idx in [0,200,400]:
    print "**************** Batch starting at", str(start_idx), "********************"
    try:
        del x_train
        del y_train
        del training_images
        del training_labels
        del training_files
    except:
        pass

    x_train, training_labels, training_files = load_training_images(start_idx, batch_size=200)
    
    if train_le == None:
        train_le = preprocessing.LabelEncoder() # Fit label encoder
        train_le.fit(training_labels)

    y_train = get_one_hot(train_le, training_labels)

    print "shallow"
    shallow.fit(x_train, y_train, batch_size=64, epochs=NUM_EPOCHS)
    
    print "six_layer"
    six_layer.fit(x_train, y_train, batch_size=64, epochs=NUM_EPOCHS)
    
    print "nine_layer"
    nine_layer.fit(x_train, y_train, batch_size=64, epochs=NUM_EPOCHS)
    
x_val, val_labels = load_validation_images()
y_val = get_one_hot(train_le, val_labels)

for model in [shallow, six_layer, nine_layer]:
    score = model.evaluate(x_val, y_val, batch_size=32)
    print""
    for x in range(0, len(model.metrics_names)):
        print str(model.metrics_names[x]) + ":" + str(score[x])
    print ""

shallow.save_weights("shallow_weights")
six_layer.save_weights("six_weights")
nine_layer.save_weights("nine_weights")

**************** Batch starting at 0 ********************
shallow
Epoch 1/10
39451/39451 [==============================] - 25s - loss: 4.9551 - acc: 0.0370    
Epoch 2/10
39451/39451 [==============================] - 18s - loss: 4.2822 - acc: 0.1180    
Epoch 3/10
39451/39451 [==============================] - 18s - loss: 3.5715 - acc: 0.2257    
Epoch 4/10
39451/39451 [==============================] - 18s - loss: 2.6120 - acc: 0.4024    
Epoch 5/10
39451/39451 [==============================] - 18s - loss: 1.4208 - acc: 0.6609    
Epoch 6/10
39451/39451 [==============================] - 18s - loss: 0.5690 - acc: 0.8675    
Epoch 7/10
39451/39451 [==============================] - 18s - loss: 0.2214 - acc: 0.9540    
Epoch 8/10
39451/39451 [==============================] - 18s - loss: 0.1029 - acc: 0.9803    
Epoch 9/10
39451/39451 [==============================] - 17s - loss: 0.0489 - acc: 0.9922    
Epoch 10/10
39451/39451 [==============================] - 17s - loss: 0.0251 -

19650/19650 [==============================] - 14s - loss: 1.4180 - acc: 0.6333    
Epoch 5/10
19650/19650 [==============================] - 14s - loss: 0.8542 - acc: 0.7617    
Epoch 6/10
19650/19650 [==============================] - 14s - loss: 0.5221 - acc: 0.8493    
Epoch 7/10
19650/19650 [==============================] - 14s - loss: 0.3888 - acc: 0.8836    
Epoch 8/10
19650/19650 [==============================] - 14s - loss: 0.3213 - acc: 0.9042    
Epoch 9/10
19650/19650 [==============================] - 14s - loss: 0.2616 - acc: 0.9233    
Epoch 10/10
9760/9832 [============================>.] - ETA: 0s
loss:6.12941059395
acc:0.1655817738

9832/9832 [==============================] - 2s     

loss:7.30971293104
acc:0.20260374288

9696/9832 [============================>.] - ETA: 0s
loss:5.95838514472
acc:0.202705451587



In [ ]:
score = shallow.evaluate(x_val, y_val, batch_size=32)
print""
for x in range(0, len(shallow.metrics_names)):
    print str(shallow.metrics_names[x]) + ":" + str(score[x])
print ""

In [ ]:
######### First Batch of 200
training_images, training_labels, training_files = load_training_images(start=0, batch_size=200)
#x_train, training_labels, training_files = randomize(training_images, training_labels, training_files)
x_train = training_images
train_le = preprocessing.LabelEncoder() # Fit label encoder
train_le.fit(training_labels)

y_train = get_one_hot(train_le, training_labels)

shallow.fit(x_train, y_train, batch_size=64, epochs=NUM_EPOCHS)

In [ ]:
#model.fit(x_train, y_train, batch_size=32, epochs=1)
######### 2nd Batch of 200
try:
    del x_train
    del y_train
    del training_images
    del training_labels
    del training_files
except:
    pass
    
try:
    print x_train
    print training_images
except:
    print "x_train and training_images not found!"

training_images, training_labels, training_files = load_training_images(start=200, batch_size=200)
print "after loading"
#x_train, training_labels, training_files = randomize(training_images, training_labels, training_files)
x_train = training_images
print "after randomize"
y_train = get_one_hot(train_le, training_labels)
print "after one_hot"

shallow.fit(x_train, y_train, batch_size=64, epochs=NUM_EPOCHS)

In [ ]:
######## 3rd Batch of 100
try:
    del x_train
    del y_train
    del training_images
    del training_labels
    del training_files
except:
    pass
    
try:
    print x_train
    print training_images
except:
    print "x_train and training_images not found!"

training_images, training_labels, training_files = load_training_images(start=400, batch_size=200)
#x_train, training_labels, training_files = randomize(training_images, training_labels, training_files)
x_train = training_images
y_train = get_one_hot(train_le, training_labels)

shallow.fit(x_train, y_train, batch_size=64, epochs=NUM_EPOCHS)

In [ ]:
x_val, val_labels = load_validation_images()
y_val = get_one_hot(train_le, val_labels)

score = model.evaluate(x_val, y_val, batch_size=32)
print""
for x in range(0, len(model.metrics_names)):
    print str(model.metrics_names[x]) + ":" + str(score[x])
print ""

score = model.evaluate(x_train, y_train, batch_size=32)
print ""

for x in range(0, len(model.metrics_names)):
    print str(model.metrics_names[x]) + ":" + str(score[x])
    
# without dropout: acc:0.177